In [11]:
import pandas as pd
import os
from scipy import stats
from prophet import Prophet
import numpy as np

In [12]:
ESKDALEMUI_data_renew = pd.read_csv('ESKDALEMUI_data_renew.csv')
ESKDALEMUI_data_renew['DATE'] = ESKDALEMUI_data_renew['DATE'].astype(int)
ESKDALEMUI_data_renew = ESKDALEMUI_data_renew[ESKDALEMUI_data_renew['DATE']<20220418]
ESKDALEMUI_data_renew['DATE'] = ESKDALEMUI_data_renew['DATE'].astype(str)
ESKDALEMUI_data_renew = ESKDALEMUI_data_renew.dropna()
ESKDALEMUI_data_renew

,DATE,CC,QQ,SS,TG,TN,TX
0,19570101,8.0,8.0,0.0,28.000000,11.0,61.000000
1,19570102,8.0,8.0,0.0,20.000000,-22.0,89.000000
2,19570103,7.0,12.0,2.0,36.000000,-17.0,100.000000
3,19570104,7.0,8.0,0.0,70.000000,39.0,100.000000
4,19570105,7.0,8.0,0.0,72.000000,44.0,72.000000
...,...,...,...,...,...,...,...
23843,20220413,8.0,82.0,6.0,102.000000,67.0,117.000000
23844,20220414,8.0,54.0,0.0,97.000000,77.0,139.000000
23845,20220415,8.0,55.0,0.0,104.000000,70.0,142.000000
23846,20220416,8.0,103.0,13.0,104.000000,67.0,184.000000


In [13]:
LONDON_data_renew = pd.read_csv('LONDON_data_renew.csv')
LONDON_data_renew['DATE'] = LONDON_data_renew['DATE'].astype(int)
LONDON_data_renew = LONDON_data_renew[LONDON_data_renew['DATE']<20200418]
LONDON_data_renew['DATE'] = LONDON_data_renew['DATE'].astype(str)
LONDON_data_renew

,DATE,CC,QQ,SS,TG,TN,TX
0,19790101,2.0,52.0,7.0,-41.0,-75.0,23.0
1,19790102,6.0,27.0,1.7,-26.0,-75.0,16.0
2,19790103,5.0,13.0,0.0,-28.0,-72.0,13.0
3,19790104,8.0,13.0,0.0,-26.0,-65.0,-3.0
4,19790105,6.0,29.0,2.0,-8.0,-14.0,56.0
...,...,...,...,...,...,...,...
15078,20200413,3.0,241.0,11.0,100.0,82.0,119.0
15079,20200414,1.0,253.0,9.9,83.0,32.0,135.0
15080,20200415,4.0,305.0,14.5,108.0,26.0,191.0
15081,20200416,7.0,289.0,13.6,121.0,46.0,196.0


In [14]:
import logging
import cmdstanpy

logging.getLogger(cmdstanpy.__name__).setLevel(logging.WARNING)

def train_model(data_train,data_test):
    data_train['ds'] = pd.to_datetime(data_train['ds'], format='%Y%m%d') 
    meanD = data_train['y'].mean()
    stdD = data_train['y'].std()
    data_train['y'] = (data_train['y'] - meanD ) / (stdD)  
    m = Prophet()
    m.add_regressor('CC')
    m.add_regressor('QQ')
    m.add_regressor('SS')
    m.fit(data_train)
    forecast = m.predict(data_test.drop(columns='y'))
    forecast['yhat'] = forecast['yhat']*stdD +  meanD 
    return forecast


def deal_data(model_data_input,input3):
    # 划分数据级
    model_data_test  = model_data_input[['DATE',input3,'CC','QQ','SS']][-30:]
    model_data_test['DATE'] = pd.to_datetime(model_data_test['DATE'], format='%Y%m%d') 
    model_data_test = model_data_test.rename(columns={'DATE':'ds', input3:'y'})

    model_data_train = model_data_input[['DATE',input3,'CC','QQ','SS']][:-30]
    model_data_train = model_data_test.rename(columns={'DATE':'ds',input3:'y'})
    model_data_train['ds'] = pd.to_datetime(model_data_train['ds'], format='%Y%m%d') 
    return model_data_train,model_data_test


def get_endData(forecast,input2):
    try:
        data = float(forecast[forecast['ds']==input2]['yhat'])/10
    except:
        data = float(forecast[forecast['ds']=='2022-3-18']['yhat'])/10
    return data

def model_test(input1,input2,input3):#input1-city input2-date input3- level
    answer = ''
    if('edinburgh' in input1):
        if('high' in  input3):
            model_data_train,model_data_test = deal_data(ESKDALEMUI_data_renew,'TX')
            forecast = train_model(model_data_train,model_data_test)
            end_data = get_endData(forecast,input2)
            # print(end_data)
            answer ='At a {} and {}, {} was {:.2f}.'.format(input1,input2,"the highest temperature",end_data)

        elif('low' in input3):
            model_data_train,model_data_test = deal_data(ESKDALEMUI_data_renew,'TN')
            forecast = train_model(model_data_train,model_data_test)
            data = float(forecast[forecast['ds']=='2022-3-19']['yhat'])
            end_data = get_endData(forecast,input2)
            answer ='At a {} and {}, {} was {:.2f}.'.format(input1,input2,"the lowest temperature",end_data)

        elif('average' in input3):
            model_data_train,model_data_test = deal_data(ESKDALEMUI_data_renew,'TG')
            forecast = train_model(model_data_train,model_data_test)
            end_data = get_endData(forecast,input2)
            # print(end_data)
            answer ='At a {} and {}, {} was {:.2f}.'.format(input1,input2,"the average temperature",end_data)
        else:
            model_test('edinburgh',input2,'high')
            model_test('edinburgh',input2,'low')
            model_test('edinburgh',input2,'average')
    elif('london' in input1):
        if('high' in  input3):
            model_data_train,model_data_test = deal_data(LONDON_data_renew,'TX')
            forecast = train_model(model_data_train,model_data_test)
            end_data = get_endData(forecast,input2)
            # print(end_data)
            answer ='At a {} and {}, {} was {:.2f}.'.format(input1,input2,"the highest temperature",end_data)

        elif('low' in input3):
            model_data_train,model_data_test = deal_data(LONDON_data_renew,'TN')
            forecast = train_model(model_data_train,model_data_test)
            end_data = get_endData(forecast,input2)
            # print(end_data)
            answer ='At a {} and {}, {} was {:.2f}.'.format(input1,input2,"the lowest temperature", end_data)

        elif('average' in input3):
            model_data_train,model_data_test = deal_data(LONDON_data_renew,'TG')
            forecast = train_model(model_data_train,model_data_test)
            end_data = get_endData(forecast,input2)
            # print(end_data)
            answer ='At a {} and {}, {} was {:.2f}.'.format(input1,input2,"the average temperature",end_data)
        else:
            model_test('london',input2,'high')
            model_test('london',input2,'low')
            model_test('london',input2,'average')
    print(answer)
    return answer
model_test('edinburgh','2022-3-20','low')
model_test('edinburgh','2022-3-20','high')
model_test('edinburgh','2022-3-20','average')
model_test('edinburgh','2022-3-20','')
model_test('london','2020-3-20','low')
model_test('london','2020-3-20','high')
model_test('london','2020-3-20','average')
model_test('london','2020-3-20','')

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.



Initial log joint probability = -15.8044
Iteration  1. Log joint probability =    13.0131. Improved by 28.8176.
Iteration  2. Log joint probability =    23.0666. Improved by 10.0534.
Iteration  3. Log joint probability =    23.0967. Improved by 0.0301122.
Iteration  4. Log joint probability =    23.1208. Improved by 0.0241573.
Iteration  5. Log joint probability =    23.1469. Improved by 0.0260185.
Iteration  6. Log joint probability =    23.1745. Improved by 0.0276212.
Iteration  7. Log joint probability =    23.1958. Improved by 0.0213315.
Iteration  8. Log joint probability =    23.2078. Improved by 0.0119759.
Iteration  9. Log joint probability =    23.2215. Improved by 0.0137389.
Iteration 10. Log joint probability =    23.2239. Improved by 0.00240492.
Iteration 11. Log joint probability =    23.2334. Improved by 0.00943727.
Iteration 12. Log joint probability =    23.2357. Improved by 0.00233232.
Iteration 13. Log joint probability =    23.2407. Improved by 0.00494929.
Iteration

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.


At a edinburgh and 2022-3-20, the lowest temperature was -1.98.

Initial log joint probability = -15.5056
Iteration  1. Log joint probability =    5.94125. Improved by 21.4469.
Iteration  2. Log joint probability =     7.3368. Improved by 1.39554.
Iteration  3. Log joint probability =    7.81782. Improved by 0.481026.
Iteration  4. Log joint probability =    7.84425. Improved by 0.0264265.
Iteration  5. Log joint probability =    8.07546. Improved by 0.231212.
Iteration  6. Log joint probability =    8.38022. Improved by 0.304759.
Iteration  7. Log joint probability =    8.40289. Improved by 0.0226736.
Iteration  8. Log joint probability =    8.41548. Improved by 0.0125874.
Iteration  9. Log joint probability =    8.42172. Improved by 0.00624345.
Iteration 10. Log joint probability =    8.43154. Improved by 0.00982049.
Iteration 11. Log joint probability =    8.43191. Improved by 0.0003705.
Iteration 12. Log joint probability =    8.43735. Improved by 0.0054397.
Iteration 13. Log joint

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.


...

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.


At a london and 2020-3-20, the highest temperature was 6.41.

Initial log joint probability = -17.7419
Iteration  1. Log joint probability =    9.30854. Improved by 27.0504.
Iteration  2. Log joint probability =    9.93887. Improved by 0.630327.
Iteration  3. Log joint probability =    13.4453. Improved by 3.50644.
Iteration  4. Log joint probability =    13.7588. Improved by 0.313511.
Iteration  5. Log joint probability =    13.7889. Improved by 0.0300546.
Iteration  6. Log joint probability =    13.8158. Improved by 0.0268868.
Iteration  7. Log joint probability =    13.8282. Improved by 0.0124124.
Iteration  8. Log joint probability =    13.8318. Improved by 0.00364191.
Iteration  9. Log joint probability =    13.8998. Improved by 0.0680001.
Iteration 10. Log joint probability =    13.9691. Improved by 0.0692664.
Iteration 11. Log joint probability =    14.0331. Improved by 0.0640472.
Iteration 12. Log joint probability =     14.065. Improved by 0.0318494.
Iteration 13. Log joint pr

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.


At a london and 2020-3-20, the lowest temperature was 4.61.

Initial log joint probability = -13.6195
Iteration  1. Log joint probability =    11.8987. Improved by 25.5182.
Iteration  2. Log joint probability =    16.6542. Improved by 4.75542.
Iteration  3. Log joint probability =    16.9609. Improved by 0.306772.
Iteration  4. Log joint probability =    17.0421. Improved by 0.0811607.
Iteration  5. Log joint probability =    17.1175. Improved by 0.0754271.
Iteration  6. Log joint probability =    17.1241. Improved by 0.00659732.
Iteration  7. Log joint probability =    17.1242. Improved by 7.12846e-05.
Iteration  8. Log joint probability =    17.1403. Improved by 0.016067.
Iteration  9. Log joint probability =    17.1588. Improved by 0.0184989.
Iteration 10. Log joint probability =    17.1685. Improved by 0.00970172.
Iteration 11. Log joint probability =    17.1718. Improved by 0.00334341.
Iteration 12. Log joint probability =    17.1747. Improved by 0.00284155.
Iteration 13. Log join

''